#### Estimate the amount of missing sediment input in Canada Basin using the turbocline depth

In [4]:
import numpy as np
import netCDF4 as nc
import glob

%matplotlib inline

##### Parameters

In [37]:
imin, imax = 1480, 2180
jmin, jmax = 160, 800
isize, jsize = imax-imin, jmax-jmin

###### Load files

In [10]:
mask  = nc.Dataset('/ocean/brogalla/GEOTRACES/ariane_runs/ANHA12_Ariane_mesh.nc')
tmask = np.array(mask.variables['tmask'])

In [11]:
file_list = glob.glob('/data/brogalla/ANHA12/ANHA12-EXH006_5d_gridT_y2015*')

##### Calculations

In [52]:
# Find indices to separate out Canada Basin:
x_ind = np.arange(1620, 2100, 1)
y_ind = (-7/8)*x_ind + 1517 + 700

CB_indx = []
CB_indy = []
for index in range(0,len(x_ind)):
    CB_x = np.arange(x_ind[index],2180,1)
    CB_y = np.ones(CB_x.shape)*y_ind[index]
    
    CB_indx = np.append(CB_x, CB_indx)
    CB_indy = np.append(CB_y, CB_indy)

In [53]:
# Separate Canada Basin:
mask_CB  = np.zeros((isize,jsize))
for i, j in zip(CB_indx, CB_indy):
    mask_CB[int(i-imin),int(j-jmin)]  = 1
    
mask_CB[150:-1 ,-8:-1] = 1

In [59]:
turbo_maxs = np.zeros((len(file_list),1))
turbo_mins = np.zeros((len(file_list),1))
turbo_aves = np.zeros((len(file_list),1))

i=0
for file in file_list:
    ds = nc.Dataset(file)
    
    # Load turbocline depth variable and mask land values
    turbo   = np.array(ds.variables['sohmld'])[0,imin:imax,jmin:jmax]
    turbo_m = np.ma.masked_where((tmask[0,0,imin:imax,jmin:jmax] < 0.1) & (mask_CB==0), turbo)
    
    turbo_maxs[i] = np.ma.amax(turbo_m)
    turbo_mins[i] = np.ma.amin(turbo_m[turbo_m > 0]) # don't want to bias by zero
    turbo_aves[i] = np.ma.mean(turbo_m)
    
    i+=1
    

In [60]:
print('Maximum turbocline depth seen in 2015: ', '{:.2f}'.format(np.amax(turbo_maxs)))
print('Minimum turbocline depth seen in 2015: ', '{:.2f}'.format(np.amin(turbo_mins)))
print('Average turbocline depth seen in 2015: ', '{:.2f}'.format(np.average(turbo_aves)))

Maximum turbocline depth seen in 2015:  152.41
Minimum turbocline depth seen in 2015:  10.00
Average turbocline depth seen in 2015:  32.67


Now, to calculate the missing contribution in grams of sediment per meter squared:

In [61]:
turbo_max = np.amax(turbo_maxs)
turbo_min = np.amin(turbo_mins)
turbo_ave = np.average(turbo_aves)

In [62]:
max_missing = (4e-9*54.938*1e3*1e3*turbo_max)/(0.274*0.65)
min_missing = (4e-9*54.938*1e3*1e3*turbo_min)/(0.274*0.65)
ave_missing = (4e-9*54.938*1e3*1e3*turbo_ave)/(0.274*0.65)

In [65]:
print('Maximum missing in 2015: ', '{:.2f}'.format(max_missing), 'g/m2')
print('Minimum missing in 2015: ', '{:.2f}'.format(min_missing), 'g/m2')
print('Average missing in 2015: ', '{:.2f}'.format(ave_missing), 'g/m2')

Maximum missing in 2015:  188.06 g/m2
Minimum missing in 2015:  12.34 g/m2
Average missing in 2015:  40.31 g/m2
